# 无人车大赛 Baseline
本baseline采用PyTorch框架，应用ModelArts的Notebook进行开发

硬件规格：本案例必须使用GPU运行，请查看[《ModelArts JupyterLab 硬件规格使用指南》](https://marketplace.huaweicloud.com/markets/aihub/article/detail/?content_id=638c55c5-816d-421c-9b5f-90c804b1fa6b)了解切换硬件规格的方法

注意：由于codelabs，三天不使用会自动清理文件
请创建ModelArts Notebook开发环境进行训练

## 获取数据和代码
下载对应代码

In [2]:
import moxing as mox
mox.file.copy_parallel('obs://ma-competitions-bj4/wrc2022/wrc/','wrc')

INFO:root:Listing OBS: 1000

INFO:root:Listing OBS: 2000

INFO:root:pid: None.	1000/2313

INFO:root:pid: None.	2000/2313


其中选手根据给定的数据集自行标注并放到wrc/data/labeled文件夹内，需要为COCO格式标注 目录如下：

--|JPEGImages

--|Visualization

--|annotations.json

## 安装依赖

In [5]:
!pip install mmcv-full==1.2.4 #安装需要一段时间，请耐心等待，大概需要20min
!pip install terminaltables
!pip install pycocotools

Looking in indexes: http://repo.myhuaweicloud.com/repository/pypi/simple



     |████████████████████████████████| 216 kB 76.1 MB/s eta 0:00:01








     |████████████████████████████████| 190 kB 22.1 MB/s eta 0:00:01



  Created wheel for mmcv-full: filename=mmcv_full-1.2.4-cp37-cp37m-linux_x86_64.whl size=18994245 sha256=3698f304498a294c40caab004f275cc73de135ed9ef56a9131cf73837c4e24ed

  Stored in directory: /home/ma-user/.cache/pip/wheels/c9/30/dc/65f87e73685a68de6b2aac7ab5773070005ac28e98db880737

Successfully built mmcv-full




You should consider upgrading via the '/home/ma-user/anaconda3/envs/PyTorch-1.4/bin/python -m pip install --upgrade pip' command.

Looking in indexes: http://repo.myhuaweicloud.com/repository/pypi/simple






You should consider upgrading via the '/home/ma-user/anaconda3/envs/PyTorch-1.4/bin/python -m pip install --upgrade pip' command.

Looking in indexes: http://repo.myhuaweicloud.com/repository/pypi/simple



     |████████████████████████████████

## 模型训练

In [3]:
%cd wrc/mmdetection/

/home/ma-user/work/wrc/mmdetection


开始训练

标注后的数据放到指定文件夹，就不会出现没有对应文件的报错

要优化模型可以修改对应参数或者使用其他算法进行训练

In [7]:
!python ./tools/train.py configs/mask_rcnn/wrc.py --work-dir "/home/ma-user/work/wrc/ckpt/"

fatal: not a git repository (or any parent up to mount point /home/ma-user)

Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).

2022-06-09 16:54:48,120 - mmdet - INFO - Environment info:

------------------------------------------------------------

sys.platform: linux

Python: 3.7.10 (default, Jun  4 2021, 14:48:32) [GCC 7.5.0]

CUDA available: True

GPU 0: Tesla V100-PCIE-32GB

CUDA_HOME: /usr/local/cuda

NVCC: Cuda compilation tools, release 10.1, V10.1.243

GCC: gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0

PyTorch: 1.4.0

PyTorch compiling details: PyTorch built with:

  - GCC 7.3

  - Intel(R) Math Kernel Library Version 2019.0.4 Product Build 20190411 for Intel(R) 64 architecture applications

  - Intel(R) MKL-DNN v0.21.1 (Git Hash 7d2fd500bc78936d1d648ca713b901012f470dbc)

  - OpenMP 201511 (a.k.a. OpenMP 4.5)

  - NNPACK is enabled

  - CUDA Runtime 10.1

  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-g

训练完成之后，对模型进行测试

需要手动将表现最好的模型，拷贝到model目录（现在的model_best.pth是预训练模型)

model目录下的推理代码仅作参考，如有需要，请自行修改

## 将模型导入到ModelArtsAI应用管理
因为文件较多，需要等待一段时间
导入完成，请前往ModelArts控制台尝试在线部署或者批量部署，成功后再提交至比赛

In [ ]:
%cd ..
from modelarts.session import Session
from modelarts.model import Model
from modelarts.config.model_config import TransformerConfig,Params
import moxing as mox
import json
import re
import traceback
import random

try:
    session = Session()
    config_path = 'model/config.json' 
    if mox.file.exists(config_path):                                        # 判断一下是否存在配置文件，如果没有则不能导入模型
        model_location =  'model'
        model_name = "wrc"
        load_dict = json.loads(mox.file.read(config_path))
        model_type = load_dict['model_type']
        re_name = '_'+str(random.randint(0,1000))
        model_name += re_name
        print("正在导入模型,模型名称：", model_name)
        model_instance = Model(
                     session, 
                     model_name=model_name,               # 模型名称
                     model_version="1.0.0",               # 模型版本
                      source_location_type='LOCAL_SOURCE',
                     source_location=model_location,      # 模型文件路径
                     model_type=model_type,               # 模型类型
                     )
        print("所有模型导入完成")
    else:
        print("模型包不存在")
    
except Exception as e:
    print("发生了一些问题，请看下面的报错信息：") 
    traceback.print_exc()
    print("模型导入失败")

INFO:root:Using MoXing-v1.17.3-43fbf97f

INFO:root:Using OBS-Python-SDK-3.20.7


/home/ma-user/work/wrc

正在导入模型,模型名称： wrc_319

modelarts-cn-north-4-72b6beb2 is existed


INFO:obs:Successfully upload file /home/ma-user/work/wrc/model/.ipynb_checkpoints to OBS modelarts-cn-north-4-72b6beb2/model-0523-233758/model

INFO:obs:Successfully upload file /home/ma-user/work/wrc/model/.ipynb_checkpoints to OBS modelarts-cn-north-4-72b6beb2/model-0523-233758/model

INFO:obs:Successfully upload file /home/ma-user/work/wrc/model/.ipynb_checkpoints to OBS modelarts-cn-north-4-72b6beb2/model-0523-233758/model

INFO:obs:Successfully upload file /home/ma-user/work/wrc/model to OBS modelarts-cn-north-4-72b6beb2/model-0523-233758

INFO:obs:Successfully upload file /home/ma-user/work/wrc/model to OBS modelarts-cn-north-4-72b6beb2/model-0523-233758

INFO:obs:Successfully upload file /home/ma-user/work/wrc/model to OBS modelarts-cn-north-4-72b6beb2/model-0523-233758

INFO:obs:Successfully upload file /home/ma-user/work/wrc/model/configs/_base_/.ipynb_checkpoints to OBS modelarts-cn-north-4-72b6beb2/model-0523-233758/model/configs/_base_

INFO:obs:Successfully upload file /ho